## Used Car Price Prediction

### 1) Problem Statement
        - This Dataset comprises used cars sold on cardekho.com in India as well as important features of these cars.
        - If user can predict the price of the car based on the input features.
        - Prediction results can be used to give a new seller the price suggestion based on market condition.

### 2) Data Collection
    
    - The Dataset is collected from scrapping from cardekho website
    - The data consist of 13 columns and 15411 rows

In [1]:
## Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
dataset = pd.read_csv('cardekho_imputated.csv', index_col=0)

In [3]:
dataset.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


### Data Cleaning

#### Handling missing values
    
    - Handling Missing Values
    - Handling Duplicates
    - Check the Data type
    - Understanding the dataset

In [4]:
## Checking null values
dataset.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
## Remove unnecessary columns
dataset.drop('car_name',axis=1, inplace=True)
dataset.drop('brand',axis=1, inplace=True)

In [6]:
dataset.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
dataset.dtypes

model                 object
vehicle_age            int64
km_driven              int64
seller_type           object
fuel_type             object
transmission_type     object
mileage              float64
engine                 int64
max_power            float64
seats                  int64
selling_price          int64
dtype: object

In [8]:
## Getting All Different types of data in the dataset
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtype != 'O']
categorical_features = [feature for feature in dataset.columns if dataset[feature].dtype == 'O']
discrete_features = [feature for feature in numerical_features if len(dataset[feature].unique()) <= 25]
continuous_features = [feature for feature in numerical_features if feature not in discrete_features]
print(f"Numerical Features: {len(numerical_features)}")
print(f"Categorical Features: {len(categorical_features)}")
print(f"Discrete Features: {len(discrete_features)}")
print(f"Continuous Features: {len(continuous_features)}")

Numerical Features: 7
Categorical Features: 4
Discrete Features: 2
Continuous Features: 5


In [9]:
## Independent and Dependent Features
from sklearn.model_selection import train_test_split
X = dataset.drop(['selling_price'],axis=1)
y = dataset['selling_price']

In [10]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


### Feature Encoding and Scaling

#### One Hot Encoding for Columns which had lesser unqiue values and not ordinal

 - One Hot Encoding is a process by which categorical variable are converted into a form that could be provided by ML algorithms to do a better job in prediction

In [11]:
len(dataset['model'].unique())

120

In [12]:
dataset['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['model'] = label_encoder.fit_transform(X['model'])

In [14]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [15]:
## creating a column transformer

numerical_features = X.select_dtypes(exclude=['object']).columns
categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_tranformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')


preprocessor = ColumnTransformer(
    [
        ("oneHotEncoder", categorical_transformer, categorical_features),
        ("standardScaler", numeric_tranformer, numerical_features)
    ], remainder='passthrough' ## to keep the columns which are not in the above transformers unchanged
)


In [16]:
X = preprocessor.fit_transform(X)

In [17]:
## Separate the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12328, 14), (3083, 14), (12328,), (3083,))

### Model Training and Model Selection

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression , Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [20]:
## Creating a Function to evaluate the model

def evaluate_model(truth, prediction):
    r2 = r2_score(truth, prediction)
    mae = mean_absolute_error(truth, prediction)
    mse = mean_squared_error(truth, prediction)
    rmse = np.sqrt(mse)
    
    # print(f"R2 Score: {r2}")
    # print(f"Mean Absolute Error: {mae}")
    # print(f"Mean Squared Error: {mse}")
    # print(f"Root Mean Squared Error: {rmse}")
    
    return r2, mae, mse, rmse

In [21]:
## Model Training and Model Selection

models = {
    "Random Forest": RandomForestRegressor(),
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "K Neighbors": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor()
}

for i in range(len(models)):
    model_name = list(models.keys())[i]
    model = models[model_name]
    
    model.fit(X_train, y_train)
    
    # Predicting the test set results
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    # Evaluating the model
    print(f"Evaluating {model_name}...")
    r2_train, mae_train, mse_train, rmse_train = evaluate_model(y_train, y_train_pred)
    r2_test, mae_test, mse_test, rmse_test = evaluate_model(y_test, y_pred)
    
    print(list(models.keys())[i])

    print("----------------------------------------------------")
    print('Model performance on training set:')
    print("----------------------------------------------------")
    print('- R2 Score: {:.4f}%'.format(r2_train * 100))
    print('- Mean Absolute Error: {:.4f}'.format(mae_train))
    print('- Mean Squared Error: {:.4f}'.format(mse_train))
    print('- Root Mean Squared Error: {:.4f}'.format(rmse_train))
    print("\n")

    print("----------------------------------------------------")
    print('\nModel performance on testing set:')
    print("----------------------------------------------------")
    print('- R2 Score: {:.4f}%'.format(r2_test * 100))
    print('- Mean Absolute Error: {:.4f}'.format(mae_test))
    print('- Mean Squared Error: {:.4f}'.format(mse_test))
    print('- Root Mean Squared Error: {:.4f}'.format(rmse_test))
    print("----------------------------------------------------")
    print("----------------------------------------------------")

    print('='*35)
    print("\n")


Evaluating Random Forest...
Random Forest
----------------------------------------------------
Model performance on training set:
----------------------------------------------------
- R2 Score: 97.4213%
- Mean Absolute Error: 40161.1459
- Mean Squared Error: 20914054056.3755
- Root Mean Squared Error: 144616.9217


----------------------------------------------------

Model performance on testing set:
----------------------------------------------------
- R2 Score: 92.9516%
- Mean Absolute Error: 102215.4335
- Mean Squared Error: 53059341236.0080
- Root Mean Squared Error: 230346.1335
----------------------------------------------------
----------------------------------------------------


Evaluating Linear Regression...
Linear Regression
----------------------------------------------------
Model performance on training set:
----------------------------------------------------
- R2 Score: 62.1772%
- Mean Absolute Error: 268101.6071
- Mean Squared Error: 306756099359.7596
- Root Mean 

In [ ]:
## Initializing parameters for Hyperparameter Tuning

knn_params = {
    'n_neighbors' : [2,3,10,20,40,50]
}

rf_params = {
    'max_depth': [5,8,25,None,10],
    'max_features': [5,7,"auto",8],
    'min_samples_leaf': [2,8,15,20],
    'n_estimators': [100,200,500,1000]

}

adaboost_param = { # base estimator = decision tree bydefault
    'n_estimators': [50,60,70,80,90],
    'loss':  ['linear', 'square', 'exponential'],
}

gbm_param = {
      'loss': ['squared_error', 'absolute_error', 'huber'],
      'criterion': ['friedman_mse', 'squared_error','mse'],
      'min_samples_split': [2, 8, 15, 10],
      'n_estimators': [100, 200, 500, 1000],
        'max_depth': [5, 8, 25, None, 10],
        'learning_rate': [0.1,0.01,0.001]

}

xgb_param = {
     'learning_rate': [0.01, 0.1],
     'max_depth': [5,8,12,20,30],
     'n_estimators': [100, 200, 300],
     'colsample_bytree': [0.5, 0.8, 1, 0.3, 0.4],
}


In [ ]:
## Model list for Hyperparameter Tuning
randomcv_models = [
    ('KNN', KNeighborsRegressor(), knn_params),
    ('Random Forest', RandomForestRegressor(), rf_params),
    ('AdaBoost', AdaBoostRegressor(), adaboost_param),
    ('Gradient Boosting', GradientBoostingRegressor(), gbm_param),
    ('XGBoost', XGBRegressor(), xgb_param)
    
]

In [25]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for model_name, model, params in randomcv_models:
    print(f"Training {model_name} with RandomizedSearchCV...")
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions= params, 
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=-1)
    
    # Fit the model
    random_search.fit(X_train, y_train)
     
    best_model = random_search.best_params_
    model_param[model_name] = best_model
    
    for model_name in model_param:
        # print(f"Best parameters for {model_name}: {model_param[model_name]}")
        print(f"Best parameters for {model_name}: {random_search.best_params_}")
        print(f"Best score for {model_name}: {random_search.best_score_}\n")

Training Random Forest with RandomizedSearchCV...
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters for Random Forest: {'n_estimators': 200, 'min_samples_leaf': 2, 'max_features': 8, 'max_depth': None}
Best score for Random Forest: 0.8458792023555995

Training XGBoost with RandomizedSearchCV...
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters for Random Forest: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.5}
Best score for Random Forest: 0.8644070823987325

Best parameters for XGBoost: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.5}
Best score for XGBoost: 0.8644070823987325



In [29]:
models = {
    "Random Forest" : RandomForestRegressor(n_estimators=100, max_depth=None, max_features='sqrt', min_samples_split=2, n_jobs=-1),
    "K-Neighbors Regressor" : KNeighborsRegressor(n_neighbors=10, n_jobs=-1),
    "AdaBoost": AdaBoostRegressor(n_estimators=60, loss='linear')
    , "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, max_depth=10, min_samples_split=8, loss='huber', criterion='friedman_mse'),
    "XGBoost": XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=300, colsample_bytree=0.5)
}

for i in range(len(list(models))):
    model_name = list(models.keys())[i]
    model = models[model_name]
    
    # Fitting the model
    model.fit(X_train, y_train)
    
    # Predicting on test data
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    
    # Evaluating the model
    print(f"Evaluating {model_name}...")
    r2_train, mae_train, mse_train, rmse_train = evaluate_model(y_train, y_train_pred)
    r2_test, mae_test, mse_test, rmse_test = evaluate_model(y_test, y_pred)
    
    print(list(models.keys())[i])

    print("----------------------------------------------------")
    print('Model performance on training set:')
    print("----------------------------------------------------")
    print('- R2 Score: {:.4f}%'.format(r2_train * 100))
    print('- Mean Absolute Error: {:.4f}'.format(mae_train))
    print('- Mean Squared Error: {:.4f}'.format(mse_train))
    print('- Root Mean Squared Error: {:.4f}'.format(rmse_train))
    print("\n")

    print("----------------------------------------------------")
    print('\nModel performance on testing set:')
    print("----------------------------------------------------")
    print('- R2 Score: {:.4f}%'.format(r2_test * 100))
    print('- Mean Absolute Error: {:.4f}'.format(mae_test))
    print('- Mean Squared Error: {:.4f}'.format(mse_test))
    print('- Root Mean Squared Error: {:.4f}'.format(rmse_test))
    print("----------------------------------------------------")
    print("----------------------------------------------------")
    
    print('='*35)
    print("\n")

   


Evaluating Random Forest...
Random Forest
----------------------------------------------------
Model performance on training set:
----------------------------------------------------
- R2 Score: 98.1211%
- Mean Absolute Error: 39688.3660
- Mean Squared Error: 15238591735.9285
- Root Mean Squared Error: 123444.6910


----------------------------------------------------

Model performance on testing set:
----------------------------------------------------
- R2 Score: 94.3602%
- Mean Absolute Error: 99135.3867
- Mean Squared Error: 42455546763.8192
- Root Mean Squared Error: 206047.4381
----------------------------------------------------
----------------------------------------------------


Evaluating K-Neighbors Regressor...
K-Neighbors Regressor
----------------------------------------------------
Model performance on training set:
----------------------------------------------------
- R2 Score: 83.7117%
- Mean Absolute Error: 103472.0474
- Mean Squared Error: 132103731766.7099
- Roo